In [1]:
import os
import pandas as pd 
from tqdm import tqdm
import re
import codecs

In [10]:
def find_total_matches(file_path):
    with open(file_path, 'r', encoding='utf-8') as file:
        text = file.read()
        occurrences = 0
        occurrences += text.count("purchase agreement")
        occurrences += text.count("purchase contract")
        return occurrences

In [4]:
def generate_directories(start_year, end_year):
    directories = []
    quarters = [1,2,3,4]
    for year in range(start_year, end_year + 1):
        for quarter in quarters:
            directory = f"sec-data/data/{year}_{quarter}/10-K"
            directories.append(directory)
    return directories


In [5]:
def find_all(a_str, sub):
    start = 0
    while True:
        start = a_str.find(sub, start)
        if start == -1: return
        yield start
        start += len(sub)
        
list(find_all('spam spam spam spam', 'spam'))

[0, 5, 10, 15]

In [7]:
def find_purchase_contract_context(file_path):
    pattern = re.compile(r'([^\.]*?purchase agreement[^\.]*\.)', re.IGNORECASE)

    # Initialize list to store contexts
    contexts = []

    with codecs.open(file_path, 'r', encoding='utf-8', errors='ignore') as file:
        text = file.read()

        instances = list(find_all(text, 'purchase agreement'))
        if len(instances) == 0:
            return contexts

        # Find matches in the text
        for instance in instances:
            start_index = max(0, instance - 500)
            end_index = min(len(text), instance + 500)
            context = text[start_index:end_index]
            contexts.append(context)

    return contexts

In [17]:
def find_purchase_all_context(file_path):

    # Initialize list to store contexts
    contexts = []

    with codecs.open(file_path, 'r', encoding='utf-8', errors='ignore') as file:
        text = file.read()

        instances = list(find_all(text, 'purchase agreement'))
        if len(instances) == 0:
            return contexts

        instances += list(find_all(text, 'purchase contract'))

        if len(instances) < 5:
            return contexts

        # Find matches in the text
        for instance in instances:
            start_index = max(0, instance - 300)
            end_index = min(len(text), instance + 300)
            context = text[start_index:end_index]
            contexts.append(context)

    return contexts

In [16]:
file = "sec-data/data/1995_1/10-K/0000893220-95-000104.txt"
find_purchase_all_context(file)



[]

In [26]:

start_year = 1995
end_year = 1995
directories = generate_directories(start_year, end_year)


csv_path = "combined.csv"
if os.path.exists(csv_path):
    existing_df = pd.read_csv(csv_path)
else:
    existing_df = pd.DataFrame()

data = []
for directory in directories:
    print(directory)
    file_list = os.listdir(directory)

    for filename in tqdm(file_list):
        file_path = os.path.join(directory, filename)
        
        if file_path.endswith(".txt"):

            contexts = find_purchase_contract_context(file_path)
            
            if len(contexts) != 0:
                for paragraph in contexts:
                    data.append({'Directory': directory, 'File': filename, 'Paragraph': paragraph})

df = pd.DataFrame(data)

combined_df = pd.concat([existing_df, df], ignore_index=True)
combined_df.to_csv(csv_path, index=True)


sec-data/data/1995_1/10-K


  9%|▉         | 118/1281 [00:18<02:58,  6.52it/s]


KeyboardInterrupt: 

In [25]:

data = []

for directory in directories:
    counter =0

    file_list = os.listdir(directory)

    for filename in tqdm(file_list):
        file_path = os.path.join(directory, filename)
        
        # Check if the file is a text file
        if file_path.endswith(".txt"):
            # Find instances of "purchase contract" in the file
            occurrences = find_purchase_contract_context(file_path)
            
            # Print the filename and the number of occurrences
            if occurrences != 0:
                print(f"File: {filename}, Occurrences: {occurrences}")
                counter += 1
                
    data.append({'Directory': directory, 'Purchase Contract Mentioned': counter, 'Total Files': len(file_list)})

df = pd.DataFrame(data)
df.to_csv("count_purchase_contract.csv", index=False)


  0%|          | 1/1281 [00:01<33:34,  1.57s/it]

File: 0000912057-95-000305.txt, Occurrences: []


  0%|          | 2/1281 [00:02<28:37,  1.34s/it]

File: 0000100122-95-000007.txt, Occurrences: []


  0%|          | 3/1281 [00:03<27:22,  1.29s/it]

...

  3%|▎         | 42/1281 [00:14<07:39,  2.70it/s]

File: 0000105982-95-000052.txt, Occurrences: ['fset by strong growth  in  higher-yielding\nloans, interest bearing time deposits due from banks\ndeclined  $962 million (71.7%) to an average of $380\nmillion in 1994. This follows a decline of $886 million\n(39.8%) in 1993 and an increase of $471 million (26.8%) in\n1992. The level of the  interbank placement  of funds\ntypically  expands  and contracts in conjunction  with the\nliquidity  and  yields  available in the  market  and\nalternative investments  for  such funds. Federal funds\nsold and   repurchase agreements averaged $471 million in\n1994, a decline of $811 million (63.3%). This follows a\ndecline of 24.9% in 1993 and an increase of 20.0%  in\n1992.  It  is expected that these  earning assets  will\nessentially remain flat during 1995.\n\nAt December 31, 1994, interest bearing time deposits due\nfrom banks totaled  $26 million, a substantial decline of\n$1.1 billion (97.8%) from the level a year earlier. At the\nsame time, f

  3%|▎         | 43/1281 [00:14<07:06,  2.90it/s]

File: 0000018540-95-000045.txt, Occurrences: ['ext  two  years  as contractual standards are met.   Additionally,\n  CSW Development-I, Inc. has entered into a fixed price contract  to\n  construct   the  Mulberry  thermal  host  facility.   The   maximum\n  potential  liability  under  this  fixed  price  contract  is   $14\n  million.  The thermal host facility is expected to be completed  by\n  the  first quarter of 1995.  CSW has provided additional guarantees\n  to the project totaling approximately $57 million.\n\n  CSWE  has  entered  into a purchase agreement  on  the  Ft.  Lupton\n  project  to provide $79.5 million of equity upon the occurrence  of\n  certain events.  As of January 9, 1995, $43 million has been  paid.\n  CSWE  has  provided three letters of credit to the project totaling\n  $14.3  million.   During March 1995, CSWE closed permanent  project\n  financing  on  the  Ft.  Lupton facility  in  the  amount  of  $208\n  million.\n\n  CSWE  has  committed to provide 

KeyboardInterrupt: 

In [14]:
import os
from pathlib import Path


path = Path("sec-data\data/2015_1/10-K/html") 
file = "edgar/data/1315257/0001193125-15-073376.txt"
a = file.split('/')

b = path / f"{a[-2]}_{a[-1]}"

b

WindowsPath('sec-data/data/2015_1/10-K/html/1315257_0001193125-15-073376.txt')